In [1]:
%config Completer.use_jedi = False #for auto complete code 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import skew
#%matplotlib inline
plt.style.use('ggplot')

def proportiondry(x):
    def mayorque(y):
        return y>0.1
    drydays=len(x)-np.sum(x.apply(mayorque))
    return float(drydays)/len(x)

def autocovariance(Xi, k=1):
    N = np.size(Xi)
    Xs = np.average(Xi)
    autoCov = 0
    for i in np.arange(0, N-k):
        autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
    return (1.0/(N-1))*autoCov

def estadisticosprec(x):
    result=[]
    a=x.dropna()
    #si se tiene menor a 10 años no contar
    if len(a)==0:
        return [None]
    result.append(np.mean(a))
    result.append(np.var(a))
    result.append(autocovariance(a))
    result.append(proportiondry(a))
    return result

#crea una tabla de los estadisticos extraidos una vez usado la fucion estadisticosprec
def crearTablaEst(location,x,names=['x','y','mean1h','var1h','autocov1h','dryperiod1h']):
    
    val=x.transpose()
    val.reset_index(drop=True, inplace=True)
    pos=location#posiciones.loc[final_station,:]
    pos.reset_index(drop=True, inplace=True)
    df=pd.concat([pos, pd.DataFrame(np.matrix(val))], axis=1)
    
    df.columns=names
    
    return df

def factores(a,b):
    if a==0.0 or b==0.0:
        return a
    else:
        return float(a)/b

In [3]:
def csv_trmm(file,month,month_hourly_date):
    month_sat_hourly=pd.read_csv(file)
    rng = month_hourly_date #bug generator
    month_hourly_date=rng[np.in1d(rng.month, month)]
    location=month_sat_hourly.iloc[:,0:2]
    location.columns=['x','y']
    month_sat_hourly=month_sat_hourly.iloc[:, 2:month_sat_hourly.shape[1]].transpose() # possible bug
    month_sat_hourly=month_sat_hourly.set_index(month_hourly_date)
    return (month_sat_hourly,location,month_hourly_date)

def common_period(dates,month,obs,daily_sat):
    daily_dates=dates[np.in1d(dates.month, month)]
    month_sat_daily=daily_sat.loc[daily_dates]
    month_obs_daily=obs.loc[daily_dates]
    
    return (month_sat_daily,month_obs_daily,daily_dates)

def filter_months_complete(percentaje,month_obs_daily,month_sat_daily,gauge_loc,month_sat_hourly):
    idx=[]
    n=percentaje*month_obs_daily.shape[0]
    
    for x in month_obs_daily.columns:
        if month_obs_daily[x].count()> n:
            idx.append(x)
            
    gauge_loc=gauge_loc.iloc[idx,:] 
    month_sat_daily=month_sat_daily.iloc[:,idx]
    month_obs_daily=month_obs_daily.iloc[:,idx]
    month_sat_hourly=month_sat_hourly.iloc[:,idx]
    
    return (gauge_loc,month_sat_daily,month_obs_daily,month_sat_hourly)   

def bias_correction(month_sat_daily,month_obs_daily,month_dates,month_sat_hourly,daily_dates):
    BIASM=[]
    for i in range(0,month_sat_daily.shape[0]):#filas
        BIAS=[]
        for j in range(0,month_sat_daily.shape[1]):#columnas
            BIAS.append(factores(month_obs_daily.iloc[i,j],month_sat_daily.iloc[i,j]))
        BIASM.append(BIAS)

        correcionM=[]
    bugM=[]
    k=0 #lleva la contabilidad de las fechas diarias
    for i in range(0,month_sat_hourly.shape[0]):
        correcion=[]
        bug=[]
        for j in range(0,month_sat_hourly.shape[1]):
            if month_dates[i].date()==daily_dates[k].date():

                bug.append(BIASM[k][j])
                correcion.append(month_sat_hourly.iloc[i,j]*BIASM[k][j])
                k=k
            else:
                k=k+1
                correcion.append(month_sat_hourly.iloc[i,j]*BIASM[k][j])

        bugM.append(bug)
        correcionM.append(correcion)
    
    month_sat_hourly_corrected=pd.DataFrame(correcionM)
    month_sat_hourly_corrected=month_sat_hourly_corrected.set_index(month_dates)
    
    return month_sat_hourly_corrected

def stats_month(month_sat_hourly_corrected,month_obs_daily,location,month):
    DATA3=month_sat_hourly_corrected.groupby(pd.Grouper(freq='3h')).agg(np.sum)
    EST3=DATA3.loc[np.in1d(DATA3.index.month, month)].dropna().apply(estadisticosprec, axis=0)
    DATA6=month_sat_hourly_corrected.groupby(pd.Grouper(freq='6h')).agg(np.sum)
    EST6=DATA6.loc[np.in1d(DATA6.index.month, month)].dropna().apply(estadisticosprec, axis=0)
    DATA12=month_sat_hourly_corrected.groupby(pd.Grouper(freq='12h')).agg(np.sum)
    EST12=DATA12.loc[np.in1d(DATA12.index.month, month)].dropna().apply(estadisticosprec, axis=0)
    DATA18=month_sat_hourly_corrected.groupby(pd.Grouper(freq='18h')).agg(np.sum)
    EST18=DATA18.loc[np.in1d(DATA18.index.month, month)].dropna().apply(estadisticosprec, axis=0)
    EST24=month_obs_daily.apply(estadisticosprec, axis=0)
    
    names=['stattion','x','y']
    agregados=[24,3,6,12,18]
    ESTS=[EST24,EST3,EST6,EST12,EST18]
    result=[]
    for i,stat in enumerate(ESTS):
        names=['x','y']
        names=names+['mean{}'.format(agregados[i]),'var{}'.format(agregados[i]),'autocov{}'.format(agregados[i]),'dryperiod{}'.format(agregados[i])]
        result.append(crearTablaEst(location,stat,names))

    df=pd.concat(result,axis=1)
    df=df.loc[:,~df.columns.duplicated()]
    return df

# OBS DAILY GAUGE STATIONS

In [4]:
rng = pd.date_range('1981-01-01','2016-12-31', freq='d')
obs=pd.read_csv('D:/Proyectos_GitHub/TrmmCorrection/data/RAIN_GAUGE.csv')
obs_daily=obs.iloc[:,3:13152].transpose()
obs_daily=obs_daily.set_index(rng)

# JAN STATS

In [30]:
rng=pd.date_range('1998-01-01 00:00:00','2014-12-31 21:00:00', freq='3H')
jan_sat_hourly,posiciones,jan_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_ENE.csv',month=1,month_hourly_date=rng)
jan_sat_daily=jan_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [31]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1998-01-01','2014-12-31', freq='d')
jan_sat_daily,jan_obs_daily,daily_dates=common_period(rng,1,obs_daily,jan_sat_daily)

In [32]:
posiciones,jan_sat_daily,jan_obs_daily,jan_sat_hourly=filter_months_complete(0.8,jan_obs_daily,jan_sat_daily,posiciones,jan_sat_hourly)

In [33]:
jan_sat_hourly_corrected=bias_correction(jan_sat_daily,jan_obs_daily,jan_hourly_date,jan_sat_hourly,daily_dates)

In [34]:
jan_stats=stats_month(jan_sat_hourly_corrected,jan_obs_daily,posiciones,month=1)
jan_stats[jan_stats.iloc[:,2:]<0]=np.NaN
jan_stats=jan_stats.dropna()

In [39]:
jan_stats.to_csv('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/data/jan_gauge_stat.csv',index=False)

In [42]:
jan_stats

,x,y,mean24,var24,autocov24,dryperiod24,mean3,var3,autocov3,dryperiod3,...,autocov6,dryperiod6,mean12,var12,autocov12,dryperiod12,mean18,var18,autocov18,dryperiod18
0,-80.53361,-3.633611,1.886072,69.095162,9.799845,0.719165,0.111788,2.874992,0.397893,0.979839,...,0.583928,0.966319,0.447154,14.869119,0.783688,0.941176,0.671368,24.576245,1.020372,0.920228
1,-80.45750,-3.813333,3.083231,109.391423,21.312226,0.561670,0.218207,6.687247,0.340124,0.961812,...,1.010593,0.935958,0.872830,30.440933,5.099824,0.895636,1.310488,51.383409,6.015125,0.847578
2,-80.45722,-3.508333,1.857495,67.992577,11.265585,0.791271,0.142149,4.758416,0.172632,0.983397,...,1.250197,0.971063,0.568596,23.731613,1.246796,0.949715,0.853704,32.019108,4.210720,0.925926
3,-80.32194,-3.440556,2.362998,91.248403,9.480408,0.722960,0.136456,5.488107,1.139370,0.980787,...,2.581907,0.966319,0.545825,23.391666,6.065467,0.944023,0.819516,51.139951,1.741574,0.925926
4,-80.66083,-3.948889,2.042952,39.920727,8.426427,0.763810,0.126122,1.780380,0.118635,0.975806,...,0.306879,0.959203,0.504488,8.417180,0.286971,0.931689,0.757450,12.210617,1.383231,0.901709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-69.99972,-17.559167,2.375750,19.102245,9.267064,0.554080,0.246784,1.613173,0.162199,0.902751,...,0.165458,0.843454,0.987135,7.474060,1.173919,0.726755,1.482090,12.733369,3.744338,0.656695
246,-69.77944,-17.525000,3.112429,20.841440,7.045815,0.440228,0.348437,2.229477,0.178550,0.877846,...,0.125546,0.805503,1.393748,9.412202,0.920831,0.657495,2.084881,15.019468,3.386869,0.571225
247,-69.62628,-17.579861,2.956632,21.040603,9.462752,0.513684,0.280930,2.154098,0.260244,0.925522,...,0.315294,0.879981,1.123719,8.995012,1.137927,0.773245,1.687179,15.593789,3.131063,0.698006
248,-69.46759,-17.393589,3.712238,25.935864,13.089671,0.449597,0.355223,2.926089,0.194460,0.917694,...,0.295933,0.861480,1.420892,11.278979,0.987613,0.736243,2.114491,19.043457,4.489936,0.668091


# FEB STATS

In [5]:
rng=pd.date_range('1999-02-01 00:00:00','2015-02-28 21:00:00', freq='3H')
feb_sat_hourly,posiciones,feb_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_FEB.csv',month=2,month_hourly_date=rng)
feb_sat_daily=feb_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [6]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1999-02-01','2015-02-28', freq='d')
feb_sat_daily,feb_obs_daily,daily_dates=common_period(rng,2,obs_daily,feb_sat_daily)
#feb_obs_daily.to_csv('D:/Proyectos_GitHub/TrmmCorrection/data/gauge_feb_data.csv',index=True,index_label='fecha')

In [14]:
posiciones,feb_sat_daily,feb_obs_daily,feb_sat_hourly=filter_months_complete(0.8,feb_obs_daily,feb_sat_daily,posiciones,feb_sat_hourly)
#feb_obs_daily.to_csv('D:/Proyectos_GitHub/TrmmCorrection/data/gauge_feb_80percent_complete_data.csv',index=True,index_label='fecha')
#posiciones.to_csv('D:/Proyectos_GitHub/TrmmCorrection/data/gauge_feb_position_80percent_complete_data.csv',index=False)

In [8]:
feb_sat_hourly_corrected=bias_correction(feb_sat_daily,feb_obs_daily,feb_hourly_date,feb_sat_hourly,daily_dates)

In [11]:
feb_stats=stats_month(feb_sat_hourly_corrected,feb_obs_daily,posiciones,month=2)

In [13]:
feb_stats.to_csv('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/data/feb_gauge_stat.csv',index=False)

In [12]:
feb_stats

,x,y,mean24,var24,autocov24,dryperiod24,mean3,var3,autocov3,dryperiod3,...,autocov6,dryperiod6,mean12,var12,autocov12,dryperiod12,mean18,var18,autocov18,dryperiod18
0,-80.53361,-3.633611,3.159336,77.801825,21.766635,0.590708,0.179063,5.386324,0.327334,0.972917,...,0.528551,0.953646,0.716250,24.543771,2.470237,0.914583,1.067757,43.983791,0.025104,0.887850
1,-80.45750,-3.813333,5.040156,154.176386,23.295429,0.483333,0.416195,10.830484,1.331110,0.935156,...,2.025190,0.894271,1.664779,55.535569,6.336754,0.825000,2.483233,83.866026,17.344001,0.763240
2,-80.45722,-3.508333,3.102296,105.140057,9.841639,0.684760,0.224661,10.258401,0.293938,0.971875,...,0.552399,0.951562,0.898646,41.694601,1.469562,0.917708,1.343452,64.954109,7.250879,0.884735
3,-80.32194,-3.440556,4.240023,126.956981,28.050210,0.629167,0.221927,6.389306,0.544053,0.974479,...,0.689582,0.957812,0.887708,29.038533,1.859142,0.926042,1.327414,48.269925,1.710575,0.904984
4,-80.66083,-3.948889,3.885179,84.833470,10.870223,0.648421,0.294797,6.337306,0.247685,0.963542,...,0.752906,0.935417,1.179188,29.747390,2.534283,0.887500,1.721838,46.416957,5.882302,0.848910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,-69.81306,-17.237222,3.991949,36.185657,15.454496,0.473418,0.302477,3.047204,0.417046,0.936979,...,0.816029,0.897396,1.209906,12.379261,2.577644,0.805208,1.779143,21.273190,6.251833,0.750779
252,-69.77944,-17.525000,3.571500,27.732492,8.831444,0.414583,0.423544,2.944728,0.283139,0.864323,...,0.482250,0.778646,1.694177,13.070041,1.381439,0.619792,2.492123,21.301323,5.352422,0.512461
253,-69.62628,-17.579861,3.776351,38.287256,14.104773,0.495495,0.364036,3.712138,0.455113,0.922656,...,0.556189,0.877083,1.456146,15.469516,1.891888,0.760417,2.152509,26.300022,6.139947,0.691589
254,-69.46759,-17.393589,4.105708,27.924056,13.166166,0.418750,0.416044,3.547367,0.301641,0.911198,...,0.322718,0.847396,1.664177,13.079315,0.620432,0.706250,2.462507,22.388120,4.246380,0.626168


# MAR STATS

In [14]:
rng=pd.date_range('1998-01-01 00:00:00','2014-12-31 21:00:00', freq='3H')
mar_sat_hourly,posiciones,mar_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_MAR.csv',month=3,month_hourly_date=rng)
mar_sat_daily=mar_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [15]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1998-01-01','2014-12-31', freq='d')
mar_sat_daily,mar_obs_daily,daily_dates=common_period(rng,3,obs_daily,mar_sat_daily)

In [16]:
posiciones,mar_sat_daily,mar_obs_daily,mar_sat_hourly=filter_months_complete(0.8,mar_obs_daily,mar_sat_daily,posiciones,mar_sat_hourly)

In [17]:
mar_sat_hourly_corrected=bias_correction(mar_sat_daily,mar_obs_daily,mar_hourly_date,mar_sat_hourly,daily_dates)

In [18]:
mar_stats=stats_month(mar_sat_hourly_corrected,mar_obs_daily,posiciones,month=3)
mar_stats[mar_stats.iloc[:,2:]<0]=np.NaN
mar_stats=mar_stats.dropna()

In [19]:
mar_stats.head()

,x,y,mean24,var24,autocov24,dryperiod24,mean3,var3,autocov3,dryperiod3,...,autocov6,dryperiod6,mean12,var12,autocov12,dryperiod12,mean18,var18,autocov18,dryperiod18
0,-80.53361,-3.633611,3.324782,99.499053,11.435901,0.641366,0.210773,6.694388,0.173035,0.975806,...,0.651541,0.957306,0.843093,29.077123,1.496383,0.925996,1.265840,40.183683,5.710891,0.891738
1,-80.45750,-3.813333,5.147083,129.898728,28.278972,0.449715,0.389587,9.106653,0.891169,0.926945,...,1.413031,0.885674,1.558349,39.933612,2.803357,0.804554,2.334330,66.366376,4.250714,0.719373
2,-80.45722,-3.508333,3.615726,130.403503,11.391685,0.669355,0.245351,8.603209,0.750348,0.968216,...,1.105159,0.949241,0.981404,43.670671,0.991990,0.908918,1.472792,60.095757,10.874898,0.873219
3,-80.32194,-3.440556,4.678790,206.815173,33.169194,0.631879,0.250213,14.616943,0.594953,0.966082,...,0.089164,0.945920,1.000854,62.740795,4.137747,0.903226,1.502422,94.526401,11.788721,0.863248
5,-80.43361,-3.966944,8.996395,278.177274,66.336327,0.430740,0.670185,19.110272,2.312613,0.919592,...,1.833971,0.876186,2.680740,85.122620,5.199990,0.788425,4.014359,137.875673,19.368234,0.710826


In [20]:
mar_stats.to_csv('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/data/mar_gauge_stat.csv',index=False)

# AGOST STATS

In [22]:
rng=pd.date_range('1998-01-01 00:00:00','2014-08-31 21:00:00', freq='3H')
ago_sat_hourly,posiciones,ago_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_AGO.csv',month=8,month_hourly_date=rng)
ago_sat_daily=ago_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [23]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1998-01-01','2014-12-31', freq='d')
ago_sat_daily,ago_obs_daily,daily_dates=common_period(rng,8,obs_daily,ago_sat_daily)
posiciones,ago_sat_daily,ago_obs_daily,ago_sat_hourly=filter_months_complete(0.8,ago_obs_daily,ago_sat_daily,posiciones,ago_sat_hourly)
ago_sat_hourly_corrected=bias_correction(ago_sat_daily,ago_obs_daily,ago_hourly_date,ago_sat_hourly,daily_dates)
ago_stats=stats_month(ago_sat_hourly_corrected,ago_obs_daily,posiciones,month=8)
ago_stats[ago_stats.iloc[:,2:]<0]=np.NaN
ago_stats=ago_stats.dropna()
ago_stats.to_csv('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/data/agost_gauge_stat.csv',index=False)

# JULY STATS

In [24]:
rng=pd.date_range('1998-01-01 00:00:00','2014-12-31 21:00:00', freq='3H')
jul_sat_hourly,posiciones,jul_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_JUL.csv',month=7,month_hourly_date=rng)
jul_sat_daily=jul_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [25]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1998-01-01','2014-12-31', freq='d')
jul_sat_daily,jul_obs_daily,daily_dates=common_period(rng,7,obs_daily,jul_sat_daily)

In [26]:
posiciones,jul_sat_daily,jul_obs_daily,jul_sat_hourly=filter_months_complete(0.8,jul_obs_daily,jul_sat_daily,posiciones,jul_sat_hourly)
jul_sat_hourly_corrected=bias_correction(jul_sat_daily,jul_obs_daily,jul_hourly_date,jul_sat_hourly,daily_dates)
jul_stats=stats_month(jul_sat_hourly_corrected,jul_obs_daily,posiciones,month=7)


In [28]:
jul_stats[jul_stats.iloc[:,2:]<0]=np.NaN
jul_stats=jul_stats.dropna()
jul_stats.to_csv('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/data/july_gauge_stat.csv',index=False)